In [2]:
import pandas as pd

In [ ]:
ea = pd.read_excel('EA.xlsx',index_col=0)

In [ ]:
stat = ea.ticker.value_counts()

In [ ]:
stat.to_csv('stat.csv')

In [3]:
script = '''[Scene: last 5 seconds: A young woman walks confidently down a bustling city street, wearing stylish blue jeans.]  
Narrator: Looking to redefine your everyday style?  

[Scene: last 5 seconds: The woman stops by a modern café, the camera focusing on the perfect fit of the jeans.]  
Narrator: Our blue jeans are designed for both comfort and elegance.  

[Scene: last 5 seconds: She sits at a café, sipping a coffee, crossing her legs which show off the jeans effortlessly.]  
Narrator: Made with the finest denim, they'll shape to your body like a second skin.  

[Scene: last 5 seconds: The woman stands up and continues her walk, catching envious glances from passersby.]  
Narrator: Ready to turn heads wherever you go?  

[Scene: last 5 seconds: The camera zooms in on her jeans, highlighting the intricate stitching and quality fabric.]  
Narrator: Discover the perfect pair of blue jeans.  

[Scene: last 5 seconds: The woman smiles confidently as she walks into the sunset, the brand logo appearing on the screen.]  
Narrator: Elevate your style—choose the best.'''

In [4]:
import re
pattern = r'\[Scene: last (\d+) seconds: (.*?)\]'

# Find all matches
matches = re.findall(pattern, script+'\n')

# Format the results as required
formatted_results = [[int(seconds), description] for seconds, description in matches]



In [5]:
pattern = r'(?:\[Narrator\]?[:]?|Narrator:)\s*([^\[]*)'
narrator_paragraphs = re.findall(pattern, script+'\n', re.DOTALL | re.MULTILINE)
script_res = '\n'.join(paragraph.strip() for paragraph in narrator_paragraphs )
sentences = script_res.splitlines()

In [6]:
sentences

['Looking to redefine your everyday style?',
 'Our blue jeans are designed for both comfort and elegance.',
 "Made with the finest denim, they'll shape to your body like a second skin.",
 'Ready to turn heads wherever you go?',
 'Discover the perfect pair of blue jeans.',
 'Elevate your style—choose the best.']

In [7]:
pattern = r'(?:\[Narrator\]?[:]?|Narrator:)\s*([^\[]*)'
narrator_paragraphs = re.findall(pattern, script+'\n', re.DOTALL | re.MULTILINE)
script = '\n'.join(paragraph.strip() for paragraph in narrator_paragraphs )

In [8]:
import requests
#import ffmpeg
# import sys
# sys.path.append('/Users/xiyuantao/Desktop/ffprobe')
from pydub import AudioSegment
import io

c:\Users\57792\anaconda3\envs\streamlit\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
model_url = "https://api.elevenlabs.io/v1/text-to-speech/Mu5jxyqZOLIGltFpfalg/with-timestamps"

payload = {
    "voice_settings": {
        "similarity_boost": 0.8,
        "stability": 0.3
    },
    "text": script
}
headers = {
    "xi-api-key": "81274497562cc26cda36b0b55d215eb2",
    "Content-Type": "application/json"
}
response_audio = requests.request("POST", model_url, json=payload, headers=headers)

In [11]:
import json
import base64
#import ffprobe

In [12]:
json_string = response_audio.content.decode("utf-8")
response_dict = json.loads(json_string)
audio_bytes = base64.b64decode(response_dict["audio_base64"])

In [ ]:
sys.path

In [8]:
from pydub import AudioSegment
import sys
sys.path.append('/Users/xiyuantao/Desktop/ffprobe')
sys.path.append('/Users/xiyuantao/Desktop/streamlit/streamlit/ffmpeg')

import io

In [13]:
original_audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")

FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'

In [ ]:
json_string = response.content.decode("utf-8")

# parse the JSON string and load the data as a dictionary
response_dict = json.loads(json_string)

In [ ]:
response_dict.content

In [ ]:
# Calculate timestamps for each sentence
sentence_timestamps = []
sentence_start_time = 0
sentences = script.splitlines()
char_index = 0

for sentence in sentences:
    print(response_dict['alignment']['characters'][char_index:char_index + len(sentence) - 1])
    sentence_start_time = response_dict['alignment']['character_start_times_seconds'][char_index]
    sentence_end_time = response_dict['alignment']['character_end_times_seconds'][char_index + len(sentence) - 1]
    duration = sentence_end_time - sentence_start_time
    sentence_timestamps.append(duration)
    char_index += len(sentence) + 1  # +1 to account for the newline character

# Upload Audio to OSS



In [ ]:
script

In [ ]:
sentence_timestamps

In [ ]:
sentence_timestamps = []
sentence_start_time = 0
sentences = script.splitlines()
char_index = 0

for sentence in sentences:
    sentence_start_time = response_dict['character_start_times_seconds'][char_index]
    sentence_end_time = response_dict['character_end_times_seconds'][char_index + len(sentence) - 1]
    duration = sentence_end_time - sentence_start_time
    sentence_timestamps.append(duration)
    char_index += len(sentence) + 1  # +1 to account for the newline character

In [ ]:
from pydub import AudioSegment
import io


pattern = r'(?:\[Narrator\]?[:]?|Narrator:)\s*([^\[]*)'
narrator_paragraphs = re.findall(pattern, st.session_state.script+'\n', re.DOTALL | re.MULTILINE)
script = '\n'.join(paragraph.strip() for paragraph in narrator_paragraphs )

# Generate Audio with elevenlabs
model_url = "https://api.elevenlabs.io/v1/text-to-speech/Mu5jxyqZOLIGltFpfalg/with-timestamps"

payload = {
    "voice_settings": {
        "similarity_boost": 0.8,
        "stability": 0.3
    },
    "text": script
}
headers = {
    "xi-api-key": st.secrets['elevenlabs_key'],
    "Content-Type": "application/json"
}
response_audio = requests.request("POST", model_url, json=payload, headers=headers)

# Error handling
if response_audio.status_code != 200:
    st.error(f"Error generating audio: {response_audio.status_code} - {response_audio.text}")
    return

response_dict = response_audio.json()

# Process audio and timestamps
audio_bytes = base64.b64decode(response_dict["audio_base64"])
original_audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")

sentence_timestamps = []
char_index = 0

new_audio = AudioSegment.silent(duration=0)  # Start with an empty audio segment

for sentence in script.splitlines():
    sentence_start_time = response_dict['character_start_times_seconds'][char_index]
    sentence_end_time = response_dict['character_end_times_seconds'][char_index + len(sentence) - 1]
    duration = sentence_end_time - sentence_start_time
    sentence_timestamps.append(duration)

    # Extract the audio for this sentence
    start_ms = sentence_start_time * 1000
    end_ms = sentence_end_time * 1000
    sentence_audio = original_audio[start_ms:end_ms]

    # Calculate the silence duration needed to make the total 5 seconds
    silence_duration = max(0, 5000 - int(duration * 1000))

    # Add sentence audio and then add the calculated silence
    new_audio += sentence_audio + AudioSegment.silent(duration=silence_duration)

    char_index += len(sentence) + 1  # +1 to account for the newline character

# Save the modified audio
audio_name = time.strftime("%Y%m%d%H%M%S") + str(random.randint(1, 100)) + '_with_silence.mp3'
new_audio.export(audio_name, format="mp3")
with open(audio_name, "rb") as f:
    oss_bucket.put_object(audio_name, f.read())

time.sleep(2)
st.session_state.audio_dict = {
    'audio_name': audio_name,
    'audio_url': oss_bucket.sign_url('GET', audio_name, 180),
    'narrator_timestamp': sentence_timestamps,
}
